# Check significance of replicated genes in ion channel system

Using hypergeometric test

In [1]:

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import pandas as pd
import random


# latex rendering of text in graphs
import matplotlib as mpl
from matplotlib import rcParams
rcParams['font.family'] = 'sans-serif'
rcParams['font.sans-serif'] = ['Arial']

sns.set_style('white')

import sys

% matplotlib inline

sns.set_style("ticks", {"xtick.major.size": 15, "ytick.major.size": 15})
plt.rcParams['svg.fonttype'] = 'none'

# Load background interactome

PCnet downloaded from ndex and parsed to networkx format 
https://ndexbio.org/#/network/f93f402c-86d4-11e7-a10d-0ac135e8bacf

In [2]:

G_pcnet = nx.read_gpickle('/Users/brinrosenthal/Documents/CCBB_tickets_data/PCnet/G_PCnet.gpickle')
print(len(G_pcnet.nodes()))
print(len(G_pcnet.edges()))

19781
2724724


# Load gene2system

In [3]:
gene2system = pd.read_excel('data/supplemental_tables_cell_systems_210416.xlsx',sheet_name='Table S10',skiprows=1,index_col='Unnamed: 0')
gene2system.head()

844    845    846    847    848    849    850    851    852  \
AC008575.1  False  False  False   True  False  False  False  False  False   
AC067968.1  False  False  False  False  False  False  False  False  False   
ACHE        False  False  False  False  False  False  False  False  False   
ACTB        False  False  False  False  False  False  False  False  False   
ACTN2       False  False  False  False  False  False  False  False   True   

              853  ...    954    955    956    957    958    959    960  \
AC008575.1  False  ...  False   True  False  False   True  False   True   
AC067968.1  False  ...  False  False  False  False  False  False  False   
ACHE        False  ...  False  False  False  False   True  False  False   
ACTB        False  ...   True  False  False   True  False  False   True   
ACTN2       False  ...  False  False  False  False   True  False   True   

              961    962   963  
AC008575.1   True   True  True  
AC067968.1  False  False  True  
ACHE         True  False  True  
ACTB        False   True  True  
ACTN2        True  False  True  

[5 rows x 120 columns]

# Load Decipher karyotype genes (damaging SNVs)

DECIPHER karyotype results come from DECIPHER v9.31 (manually aqcuired from the webtool), and include only damaging SNVs/indels (no CNVs). We were not able to collect CNV information from the DECIPHER karyotype data.

Can't use CNVs because we don't know how big the full set is 


In [4]:

dec_kary_shared = pd.read_excel('data/supplemental_tables_cell_systems_210416.xlsx',sheet_name='Table S6',
                               skiprows=1)

dec_kary_shared.index=dec_kary_shared['gene']
print(len(dec_kary_shared))
dec_kary_shared.head()

437


chr    gene  num_SNVs_indels  num_likely_LOF  num_PC  \
gene                                                          
GNB1     1    GNB1                1             NaN     1.0   
CAMTA1   1  CAMTA1                1             1.0     NaN   
RERE     1    RERE                3             NaN     3.0   
MTHFR    1   MTHFR                1             NaN     1.0   
MTOR     1    MTOR                1             NaN     1.0   

        num_annotated_reg_region  num_UTR  num_ncRNA  num_other  
gene                                                             
GNB1                         NaN      NaN        NaN        NaN  
CAMTA1                       NaN      NaN        NaN        NaN  
RERE                         NaN      NaN        NaN        NaN  
MTHFR                        NaN      NaN        NaN        NaN  
MTOR                         NaN      NaN        NaN        NaN

# hypergeometric test

In [5]:
from scipy.stats import hypergeom

In [6]:
M=len(G_pcnet.nodes())
print(M)
dec_genes = np.intersect1d(dec_kary_shared['gene'].tolist(),G_pcnet.nodes())
focal_system=953 # system 953 is ion transport
focal_system_genes=gene2system[gene2system[focal_system]].index.tolist()
n=len(dec_genes)
N=len(focal_system_genes)
x=len(np.intersect1d(dec_genes,focal_system_genes))

print(n)
print(N)
print(x)

print(hypergeom.sf(x,M,n,N))

19781
427
134
13
1.251593103774936e-06
